In [ ]:
from google.colab import drive
drive.mount('/content/drive')

In [ ]:
cd drive

In [ ]:
cd "My Drive/Practice"

In [104]:
#importing modules
import numpy as np    #numpy for mathematical operation on data
import pandas as pd    # python library to work on data
import datetime         
import os            #module of python provides function for interacting with the operating syestem.
import time             
import matplotlib.pyplot as plt   #for plotting and visual representation of data
import seaborn as sns             #visualisation library based on matplotlib
import pickle
  #plotting commands just below the code cell 
%matplotlib inline

In [83]:
#downloading the saved dataset in Recommender Model
smd = pd.read_csv('smd.csv')

In [ ]:
indices = pd.Series(smd.index, index=smd['title'])

In [85]:
#recommend those which are movies similar to the input
def get_recommendations(title, cosine_sim):
    # Get the index of the movie that matches the title
    idx = indices[title]

    # Get the pairwsie similarity scores of all movies with that movie
    sim_scores = list(enumerate(cosine_sim[idx]))

    # Sort the movies based on the similarity scores
    sim_scores = sorted(sim_scores, key=lambda x: x[1], reverse=True)

    # Get the scores of the 10 most similar movies
    sim_scores = sim_scores[1:11]

    # Get the movie indices
    movie_indices = [i[0] for i in sim_scores]

    # Return the top 10 most similar movies
    return smd['title'].iloc[movie_indices]

In [86]:
filename = 'cosine_similarity.sav'
#loading the cosine_similarity matrix stored using pickle
cosine_sim_x = pickle.load(open(filename, 'rb'))

In [87]:
get_recommendations('The Dark Knight',cosine_sim_x)

8031         The Dark Knight Rises
6218                 Batman Begins
7659    Batman: Under the Red Hood
1260                Batman & Robin
1134                Batman Returns
524                         Batman
5809                   Point Blank
6645                   Harsh Times
7561                   Harry Brown
7582                      Defendor
Name: title, dtype: object

In [88]:

def convert_int(x):
  try:
    return int(x)
  except:
    return np.nan


In [89]:
rating=pd.read_csv('ratings_small.csv');
rating

,userId,movieId,rating,timestamp
0,1,31,2.5,1260759144
1,1,1029,3.0,1260759179
2,1,1061,3.0,1260759182
3,1,1129,2.0,1260759185
4,1,1172,4.0,1260759205
...,...,...,...,...
99999,671,6268,2.5,1065579370
100000,671,6269,4.0,1065149201
100001,671,6365,4.0,1070940363
100002,671,6385,2.5,1070979663


In [103]:
users=set(rating['userId'])
#set of all users 
print(len(users))

671


In [67]:
#reading links_small
id_map = pd.read_csv('links_small.csv')[['movieId', 'tmdbId']]

#changed to int (tmdb Id)
id_map['tmdbId'] = id_map['tmdbId'].apply(convert_int)
id_map.columns = ['movieId', 'id']

#merging id_map with smd on tmdbid of movie
id_map = id_map.merge(smd[['title', 'id']], on='id').set_index('title')
#id_map = id_map.set_index('tmdbId')

In [68]:
indices_map = id_map.set_index('id')

In [102]:
titles=set(smd['title']);
#set of unique movie titles
print(len(titles));

8809


In [ ]:
!pip install surprise

In [91]:
filename = 'finalized_model.sav'
#loading the SVD Model stored using pickle
algo = pickle.load(open(filename, 'rb'))

In [92]:
#hybrid recommender - using content based and collaborative filtering
#input :- user Id and Title of the Movie
#output :-similar movies sorted on the basis of expected rating by that particular user
def hybrid(userId, title):
    if(userId not in users):
      p=1;
      return p;
    if(title not in titles):
      p=2;
      return p;  
           
    idx = indices[title]
    tmdbId = id_map.loc[title]['id']
    #print(idx)
    movie_id = id_map.loc[title]['movieId']
    sim_scores = list(enumerate(cosine_sim_x[int(idx)]))
    sim_scores = sorted(sim_scores, key=lambda x: x[1], reverse=True)
    sim_scores = sim_scores[1:50]
    movie_indices = [i[0] for i in sim_scores]
    
    movies = smd.iloc[movie_indices][['title', 'vote_count', 'vote_average', 'id']]
    movies['est'] = movies['id'].apply(lambda x: algo.predict(userId, indices_map.loc[x]['movieId']).est)
    movies = movies.sort_values('est', ascending=False)
    return movies.head(10)

In [93]:
p=0;
movies=hybrid(704554242454, 'Blue');
if(movies==1):
  print("no such userId exists");
elif(movies==2):
  print("no such movie exists");
else:
  print(movies);      

no such userId exists


In [ ]:
!pip install flask-ngrok

In [95]:
#importing modules for Back-end
import numpy as np
from flask import Flask, request, jsonify, render_template
from flask_ngrok import run_with_ngrok

In [101]:
app = Flask(__name__)
#use for running in Google Colab
run_with_ngrok(app)

#Home-Page
@app.route('/')
def home():
    return render_template('index.html')

#Prediction
@app.route('/predict',methods=['POST'])
def predict():
    '''
    For rendering results on HTML GUI
    '''
    userId = int(request.form.get("userID"))
    movie_name = request.form.get("Movie_Title")
    result = hybrid(userId,movie_name)

    if(userId not in users):
      return render_template('error1.html');   
    if(movie_name not in titles):
        return render_template('error2.html');

    return render_template('results.html', tables=[result.to_html(classes='data',header=True)])

if __name__ == "__main__":
    app.run()

 * Serving Flask app "__main__" (lazy loading)
 * Environment: production
   Use a production WSGI server instead.
 * Debug mode: off


 * Running on http://127.0.0.1:5000/ (Press CTRL+C to quit)


 * Running on http://4ae1cf9f5949.ngrok.io
 * Traffic stats available on http://127.0.0.1:4040


127.0.0.1 - - [23/Aug/2020 19:30:28] "GET / HTTP/1.1" 200 -
127.0.0.1 - - [23/Aug/2020 19:30:28] "GET /static/style.css HTTP/1.1" 200 -
127.0.0.1 - - [23/Aug/2020 19:30:30] "GET /static/1512217.jpg HTTP/1.1" 200 -
127.0.0.1 - - [23/Aug/2020 19:30:45] "POST /predict HTTP/1.1" 200 -
127.0.0.1 - - [23/Aug/2020 19:30:46] "GET /static/resultStyle.css HTTP/1.1" 200 -
127.0.0.1 - - [23/Aug/2020 19:30:47] "GET /favicon.ico HTTP/1.1" 404 -
127.0.0.1 - - [23/Aug/2020 19:30:49] "GET /static/1512217.jpg HTTP/1.1" 200 -
127.0.0.1 - - [23/Aug/2020 19:31:14] "POST /predict HTTP/1.1" 200 -
127.0.0.1 - - [23/Aug/2020 19:31:17] "GET /static/1512217.jpg HTTP/1.1" 200 -
127.0.0.1 - - [23/Aug/2020 19:31:24] "POST /predict HTTP/1.1" 200 -
127.0.0.1 - - [23/Aug/2020 19:33:25] "GET /static/1512217.jpg HTTP/1.1" 200 -
[2020-08-23 19:33:30,178] ERROR in app: Exception on /predict [POST]
Traceback (most recent call last):
  File "/usr/local/lib/python3.6/dist-packages/flask/app.py", line 2447, in wsgi_app
    re